In [2]:
import json
import time
from datetime import datetime
from pathlib import Path
from typing import Optional, Dict, Any, List
import logging

class ResultLogger:
    def __init__(self, results_directory: str = "results"):
        self.results_dir = Path(results_directory)
        self.results_dir.mkdir(exist_ok=True)
        
        # Single file to store all results
        self.results_file = self.results_dir / "all_results.json"
        
    def save_result(self, 
                   member_name: str,
                   student_id: str, 
                   score: float,
                   runtime: float,
                   timestamp: Optional[str] = None,
                   algorithm_details: Dict[str, Any] = None) -> bool:
        """
        Save verification result by appending to accumulated results.
        """
        try:
            if timestamp is None:
                timestamp = datetime.now().isoformat()
            
            # Create result data with additional metadata
            result_data = {
                "member": member_name,
                "student_id": student_id,
                "score": round(score, 4),
                "runtime": round(runtime, 4),
                "timestamp": timestamp,
                "test_run": self._get_next_run_number(member_name),
                "algorithm_details": algorithm_details or {}
            }
            
            # Load existing results or create new list
            results = self._load_all_results()
            results.append(result_data)
            
            # Save back to file
            with open(self.results_file, 'w', encoding='utf-8') as f:
                json.dump(results, f, indent=2, ensure_ascii=False)
            
            print(f"✓ Result saved for {member_name}: Run #{result_data['test_run']}, Score={score:.3f}, Runtime={runtime:.3f}s")
            return True
            
        except Exception as e:
            print(f"✗ Failed to save result for {member_name}: {e}")
            return False
    
    def _load_all_results(self) -> List[Dict]:
        """Load all existing results"""
        if self.results_file.exists():
            try:
                with open(self.results_file, 'r', encoding='utf-8') as f:
                    return json.load(f)
            except (json.JSONDecodeError, FileNotFoundError):
                return []
        return []
    
    def _get_next_run_number(self, member_name: str) -> int:
        """Get the next run number for a specific member"""
        results = self._load_all_results()
        member_results = [r for r in results if r.get('member') == member_name]
        return len(member_results) + 1


result_logger = ResultLogger()


# Cell 1: Imports and Configuration
import cv2
from pyzbar.pyzbar import decode
import os
import sys
import json
from datetime import datetime
import numpy as np
import hashlib
import time
from typing import Tuple, List, Optional, Dict, Any
import logging
from dataclasses import dataclass
from pathlib import Path

# --- NEW: dlib import ---
import dlib

# GUI Specific Imports
import customtkinter as ctk
from PIL import Image, ImageTk
from tkinter import messagebox

import pyttsx3
import threading
from tkinter import Toplevel
from queue import Queue, Empty


@dataclass
class Config:
    FACE_DB_PATH: str = "student_faces"
    LOG_FILE: str = "verification_log.json"
    STUDENT_DATA_FILE: str = "student_database.json"
    SUPPORTED_EXTENSIONS: List[str] = None
    
    # Camera settings
    CAMERA_WIDTH: int = 640
    CAMERA_HEIGHT: int = 480
    CAMERA_FPS: int = 25
    
    # --- MODIFIED: Verification thresholds for dlib + LBPH ---
    LBPH_CONFIDENCE_THRESHOLD: float = 100.0 # Lower is better. This is the max distance allowed.
    VERIFICATION_SCORE_THRESHOLD: float = 0.55 # Converted score (0.0-1.0). Higher is better.
    REQUIRED_CONSECUTIVE: int = 4 # Increased for more stability
    
    # Quality thresholds
    MIN_FACE_SIZE: int = 80
    PREFERRED_FACE_SIZE: int = 120
    MIN_PHOTOS_REQUIRED: int = 5
    DEFAULT_PHOTOS_COUNT: int = 12

    # Timeout settings
    VERIFICATION_TIMEOUT: int = 30  # seconds
    QR_SCAN_TIMEOUT: int = 15      # seconds
    
    def __post_init__(self):
        if self.SUPPORTED_EXTENSIONS is None:
            self.SUPPORTED_EXTENSIONS = ['.jpg', '.jpeg', '.png', '.bmp']

# Setup logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('system.log'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

class StudentSystem:
    def __init__(self, config: Config = None):
        self.config = config or Config()
        self._setup_directories()
        self.load_student_data()
        
        # --- MODIFIED: Initialize dlib and LBPH ---
        self.dlib_detector = dlib.get_frontal_face_detector()
        self.lbph_recognizer = None # Will be created and trained on-the-fly for each student
        
        self.verification_stats = {
            'total_attempts': 0, 'successful_verifications': 0,
            'failed_verifications': 0, 'avg_verification_time': 0.0
        }
        
        logger.info("Student Verification System backend initialized with dlib and LBPH")

    def _setup_directories(self):
        Path(self.config.FACE_DB_PATH).mkdir(exist_ok=True)

    def load_student_data(self):
        try:
            if Path(self.config.STUDENT_DATA_FILE).exists():
                with open(self.config.STUDENT_DATA_FILE, 'r', encoding='utf-8') as f:
                    self.student_data = json.load(f)
                logger.info(f"Loaded data for {len(self.student_data)} students")
            else:
                self.student_data = {}
                logger.info("No existing student data found, starting fresh")
        except (json.JSONDecodeError, IOError) as e:
            logger.error(f"Error loading student data: {e}")
            self.student_data = {}

    def get_student_info(self, student_id: str) -> Optional[Dict[str, Any]]:
        return self.student_data.get(student_id)

    def log_verification(self, student_id: str, name: str, program: str, success: bool, 
                         timestamp: str, score: float = 0.0, verification_time: float = 0.0):
        log_entry = {
            "student_id": student_id, "name": name, "program": program, "success": success,
            "timestamp": timestamp, "score": score, "verification_time": verification_time,
            "session_id": hashlib.md5(f"{timestamp}{student_id}".encode()).hexdigest()[:8]
        }
        try:
            logs = []
            if Path(self.config.LOG_FILE).exists():
                with open(self.config.LOG_FILE, 'r', encoding='utf-8') as f:
                    logs = json.load(f)
            logs.append(log_entry)
            if len(logs) > 1000:
                logs = logs[-1000:]
            with open(self.config.LOG_FILE, 'w', encoding='utf-8') as f:
                json.dump(logs, f, indent=2, ensure_ascii=False)
            status = "SUCCESS" if success else "FAILED"
            logger.info(f"Verification logged: {name} - {status} (Score: {score:.3f})")
        except IOError as e:
            logger.error(f"Error logging verification: {e}")

    def view_logs(self) -> str:
        if not Path(self.config.LOG_FILE).exists():
            return "📝 No verification logs found."
        try:
            with open(self.config.LOG_FILE, 'r', encoding='utf-8') as f:
                logs = json.load(f)
            if not logs:
                return "📝 No verification logs found."
            
            header = f"{'Timestamp':<20} {'Student ID':<12} {'Name':<25} {'Status':<10} {'Score':<8} {'Time(s)':<8}\n"
            divider = "=" * 90 + "\n"
            report = "📊 Recent Verification Logs\n" + divider + header + divider
            
            for log in reversed(logs[-50:]): # Show newest logs first
                try:
                    timestamp = datetime.fromisoformat(log['timestamp']).strftime("%Y-%m-%d %H:%M:%S")
                except (TypeError, ValueError):
                    timestamp = "INVALID TIMESTAMP"
    
                status = "✅ SUCCESS" if log.get('success', False) else "❌ FAILED"
                score = f"{log.get('score', 0)*100:.1f}%"
                time_taken = f"{log.get('verification_time', 0):.1f}"
                name = log.get('name', 'N/A')[:24]
                student_id = log.get('student_id', 'N/A')
                
                report += f"{timestamp:<20} {student_id:<12} {name:<25} {status:<10} {score:<8} {time_taken:<8}\n"
    
            total = len(logs)
            successful = sum(1 for log in logs if log.get('success'))
            success_rate = (successful / total * 100) if total > 0 else 0
            report += divider + f"📈 Summary: {successful}/{total} successful ({success_rate:.1f}% success rate)"
            return report
        except Exception as e:
            logger.error(f"Error reading logs: {e}")
            return f"❌ Error reading logs: {e}"

    def validate_qr_data(self, qr_data: str) -> Tuple[Optional[str], Optional[str], Optional[str]]:
        student_id = qr_data.strip()
        student_info = self.get_student_info(student_id)
        if student_info:
            name = student_info.get("name", "N/A")
            program = student_info.get("program", "N/A")
            logger.info(f"QR ID '{student_id}' found in database for student: {name}")
            return student_id, name, program
        else:
            logger.warning(f"QR ID '{student_id}' not found in the student database.")
            return None, None, None

    def load_registered_faces(self, student_id: str) -> List[np.ndarray]:
        folder = Path(self.config.FACE_DB_PATH) / student_id
        if not folder.is_dir(): return []
        images = []
        for file_path in folder.glob("*"):
            if file_path.suffix.lower() in self.config.SUPPORTED_EXTENSIONS:
                try:
                    img = cv2.imread(str(file_path))
                    if img is not None and img.size > 0:
                        images.append(img)
                except Exception as e:
                    logger.warning(f"Failed to load image {file_path}: {e}")
        logger.info(f"Loaded {len(images)} face images for student {student_id}")
        return images

    def preprocess_face(self, face_img: np.ndarray) -> np.ndarray:
        """Simplified preprocessing for LBPH."""
        if len(face_img.shape) == 3:
            gray = cv2.cvtColor(face_img, cv2.COLOR_BGR2GRAY)
        else:
            gray = face_img.copy()

        # Resize to a standard size for consistency
        resized = cv2.resize(gray, (200, 200), interpolation=cv2.INTER_LANCZOS4)
        
        # Apply histogram equalization to improve contrast and normalize brightness
        equalized = cv2.equalizeHist(resized)
        
        return equalized

    def train_student_lbph_model(self, student_id: str) -> bool:
        """
        Trains a temporary LBPH model for a specific student using their registered faces.
        """
        logger.info(f"Starting LBPH model training for student: {student_id}")
        registered_images = self.load_registered_faces(student_id)
        if len(registered_images) < self.config.MIN_PHOTOS_REQUIRED:
            logger.error(f"Not enough registered photos to train model for {student_id}. Found {len(registered_images)}, need {self.config.MIN_PHOTOS_REQUIRED}.")
            return False

        face_samples = []
        # We assign a single, static label because we are only verifying this one person.
        student_label = 1 
        labels = []

        for img in registered_images:
            gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            # Use dlib to find the face in the registered photo
            detected_faces = self.dlib_detector(gray_img, 1)

            if detected_faces:
                # Assume the largest face is the correct one
                face = max(detected_faces, key=lambda rect: rect.width() * rect.height())
                x, y, w, h = face.left(), face.top(), face.width(), face.height()
                face_roi = gray_img[y:y+h, x:x+w]
                
                # Preprocess and add to our training data
                face_samples.append(self.preprocess_face(face_roi))
                labels.append(student_label)
        
        if not face_samples:
            logger.error(f"Could not detect any faces in the registered images for {student_id}.")
            return False
            
        logger.info(f"Training LBPH model with {len(face_samples)} face samples.")
        # Create and train the LBPH recognizer
        self.lbph_recognizer = cv2.face.LBPHFaceRecognizer_create(radius=1, neighbors=8, grid_x=8, grid_y=8)
        self.lbph_recognizer.train(face_samples, np.array(labels))
        logger.info(f"LBPH model for {student_id} trained successfully.")
        
        return True

    def export_verification_report(self) -> str:
        if not Path(self.config.LOG_FILE).exists():
            return "No verification logs found to export."
        try:
            with open(self.config.LOG_FILE, 'r', encoding='utf-8') as f: logs = json.load(f)
            if not logs: return "No verification data to export."
            
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            report_file = f"verification_report_{timestamp}.json"
            
            total = len(logs)
            successful = sum(1 for log in logs if log['success'])
            success_rate = (successful / total * 100) if total > 0 else 0
            
            report = {
                'generated_at': datetime.now().isoformat(),
                'summary': {
                    'total_attempts': total,
                    'successful_verifications': successful,
                    'success_rate_percentage': round(success_rate, 2),
                },
                'recent_logs': logs[-50:]
            }
            with open(report_file, 'w', encoding='utf-8') as f:
                json.dump(report, f, indent=2, ensure_ascii=False)
            return f"Report successfully exported to {report_file}"
        except Exception as e:
            logger.error(f"Error generating report: {e}")
            return f"Failed to generate report: {e}"

class RobustTTS:
    def __init__(self):
        self.speech_queue = Queue()
        self.worker_thread = threading.Thread(target=self._worker_loop, daemon=True)
        self.worker_thread.start()
        logger.info("RobustTTS worker thread started.")

    def _worker_loop(self):
        while True:
            try:
                text = self.speech_queue.get()
                if text is None: break

                engine = pyttsx3.init(driverName='sapi5')
                voices = engine.getProperty('voices')
                if voices:
                    for voice in voices:
                        if 'female' in voice.name.lower() or 'zira' in voice.name.lower():
                            engine.setProperty('voice', voice.id)
                            break
                engine.setProperty('rate', 155)
                engine.setProperty('volume', 0.9)

                engine.say(text)
                engine.runAndWait()
                del engine
            except Exception as e:
                logger.error(f"TTS worker loop error: {e}")

    def speak(self, text: str):
        if text and text.strip():
            while not self.speech_queue.empty():
                try: self.speech_queue.get_nowait()
                except Empty: break
            self.speech_queue.put(text.strip())

    def cleanup(self):
        logger.info("Cleaning up TTS system...")
        self.speech_queue.put(None)
        self.worker_thread.join(timeout=2.0)
        logger.info("TTS system cleaned up.")

class App(ctk.CTk):
    def __init__(self, student_system: StudentSystem):
        super().__init__()
        self.system = student_system
        self.tts_system = RobustTTS()
        self.member_name = "Member1_Dlib_LBPH"

        self.title("🎓 Student Verification System (dlib + LBPH)")
        self.geometry("1100x720")
        ctk.set_appearance_mode("Dark")
        ctk.set_default_color_theme("blue")
        
        # --- App State ---
        self.cap = None
        self.verification_active = False
        self.current_student_id = None
        self.current_student_name = None
        self.current_student_program = None
        self.consecutive_matches = 0
        self.best_score = 0.0
        self.last_known_face = None
        self.frame_count = 0
        self.verification_step = "idle" # States: idle, qr_scan, face_verify

        self.verification_start_time = None
        self.qr_scan_start_time = None
        self.max_verification_time = self.system.config.VERIFICATION_TIMEOUT
        self.max_qr_scan_time = self.system.config.QR_SCAN_TIMEOUT
        
        # --- Layout ---
        self.grid_columnconfigure(1, weight=1)
        self.grid_rowconfigure(0, weight=1)

        # --- Sidebar Frame ---
        self.sidebar_frame = ctk.CTkFrame(self, width=200, corner_radius=0)
        self.sidebar_frame.grid(row=0, column=0, rowspan=4, sticky="nsew")
        self.sidebar_frame.grid_rowconfigure(5, weight=1)
        
        self.logo_label = ctk.CTkLabel(self.sidebar_frame, text="Main Menu", font=ctk.CTkFont(size=20, weight="bold"))
        self.logo_label.grid(row=0, column=0, padx=20, pady=(20, 10))
        
        self.verify_button = ctk.CTkButton(self.sidebar_frame, text="Verify Student", command=self.verify_student_frame_event)
        self.verify_button.grid(row=1, column=0, padx=20, pady=10)

        self.logs_button = ctk.CTkButton(self.sidebar_frame, text="View Logs", command=self.view_logs_frame_event)
        self.logs_button.grid(row=2, column=0, padx=20, pady=10)

        self.export_button = ctk.CTkButton(self.sidebar_frame, text="Export Report", command=self.export_report_event)
        self.export_button.grid(row=3, column=0, padx=20, pady=10)

        self.exit_button = ctk.CTkButton(self.sidebar_frame, text="Exit", fg_color="transparent", border_width=2, text_color=("gray10", "#DCE4EE"), command=self.on_closing)
        self.exit_button.grid(row=6, column=0, padx=20, pady=20, sticky="s")
        
        # --- Verification Frame ---
        self.verify_frame = ctk.CTkFrame(self, corner_radius=0, fg_color="transparent")
        self.verify_frame.grid_columnconfigure(0, weight=1)
        
        self.camera_label = ctk.CTkLabel(self.verify_frame, text="")
        self.camera_label.grid(row=0, column=0, padx=10, pady=10)

        self.status_label = ctk.CTkLabel(self.verify_frame, text="Welcome! Click 'Start Verification' to begin.", font=ctk.CTkFont(size=16))
        self.status_label.grid(row=1, column=0, padx=10, pady=5)
        
        self.info_label = ctk.CTkLabel(self.verify_frame, text="", font=ctk.CTkFont(size=14))
        self.info_label.grid(row=2, column=0, padx=10, pady=5)
        
        self.progress_bar = ctk.CTkProgressBar(self.verify_frame, width=300)
        self.progress_bar.set(0)
        self.progress_bar.grid(row=3, column=0, padx=10, pady=5)

        self.start_button = ctk.CTkButton(self.verify_frame, text="Start Verification", command=self.start_verification_process)
        self.start_button.grid(row=4, column=0, padx=10, pady=10)

        # --- Logs Frame ---
        self.logs_frame = ctk.CTkFrame(self, corner_radius=0, fg_color="transparent")
        self.logs_frame.grid_columnconfigure(0, weight=1)
        self.logs_frame.grid_rowconfigure(0, weight=1)
        self.log_textbox = ctk.CTkTextbox(self.logs_frame, width=700, font=("Courier New", 12))
        self.log_textbox.grid(row=0, column=0, padx=20, pady=20, sticky="nsew")
        
        # --- Initial Setup ---
        self.select_frame_by_name("verify")
        self.protocol("WM_DELETE_WINDOW", self.on_closing)

    def speak_text(self, text):
        self.tts_system.speak(text)

    def show_student_details_screen(self, student_id: str, verification_score: float):
        student_info = self.system.get_student_info(student_id)
        if not student_info:
            logger.error(f"No student info found for ID: {student_id}")
            return
        
        details_window = Toplevel(self)
        details_window.title("✅ Verification Successful")
        details_window.geometry("600x500")
        details_window.configure(bg='#1a1a1a')
        details_window.resizable(False, False)
        
        details_window.transient(self)
        details_window.grab_set()
        
        details_window.update_idletasks()
        x = (details_window.winfo_screenwidth() // 2) - (600 // 2)
        y = (details_window.winfo_screenheight() // 2) - (500 // 2)
        details_window.geometry(f"600x500+{x}+{y}")
        
        main_frame = ctk.CTkFrame(details_window, fg_color="transparent")
        main_frame.pack(fill="both", expand=True, padx=20, pady=20)
        
        success_label = ctk.CTkLabel(main_frame, text="🎉 VERIFICATION SUCCESSFUL 🎉", font=ctk.CTkFont(size=24, weight="bold"), text_color="#00ff00")
        success_label.pack(pady=(0, 20))
        
        photo_frame = ctk.CTkFrame(main_frame, width=180, height=200, fg_color="#2b2b2b")
        photo_frame.pack(pady=(0, 20))
        photo_frame.pack_propagate(False)
        
        photo_loaded = False
        try:
            card_photos_folder = Path("card_photos")
            for ext in ['.jpg', '.jpeg', '.png', '.bmp']:
                photo_path = card_photos_folder / f"{student_id}{ext}"
                if photo_path.exists():
                    img = cv2.imread(str(photo_path))
                    if img is not None:
                        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                        img_pil = Image.fromarray(img_rgb).resize((170, 190), Image.Resampling.LANCZOS)
                        photo_ctk = ctk.CTkImage(light_image=img_pil, dark_image=img_pil, size=(170, 190))
                        photo_label = ctk.CTkLabel(photo_frame, image=photo_ctk, text="")
                        photo_label.pack(expand=True, padx=5, pady=5)
                        photo_loaded = True
                        break
        except Exception as e: logger.warning(f"Could not load student photo: {e}")
        
        if not photo_loaded:
            photo_label = ctk.CTkLabel(photo_frame, text="👤\nNo Photo\nAvailable", font=ctk.CTkFont(size=14), text_color="#cccccc")
            photo_label.pack(expand=True)
        
        details_frame = ctk.CTkFrame(main_frame, fg_color="#2b2b2b")
        details_frame.pack(fill="x", pady=(0, 20))
        
        name = student_info.get("name", "N/A")
        program = student_info.get("program", "N/A")
        batch = student_info.get("batch", "N/A")
        
        details_data = [("👤 Name:", name), ("📚 Program:", program), ("🎓 Batch:", f"20{batch}" if batch.isdigit() else batch), ("🔍 Verification Score:", f"{verification_score*100:.1f}%")]
        
        for i, (label, value) in enumerate(details_data):
            detail_frame = ctk.CTkFrame(details_frame, fg_color="transparent")
            detail_frame.pack(fill="x", padx=20, pady=5)
            ctk.CTkLabel(detail_frame, text=label, font=ctk.CTkFont(size=14, weight="bold"), anchor="w").pack(side="left", fill="x", expand=True)
            ctk.CTkLabel(detail_frame, text=str(value), font=ctk.CTkFont(size=14), text_color="#00bfff", anchor="e").pack(side="right")
        
        ctk.CTkButton(main_frame, text="✓ Close", command=details_window.destroy, fg_color="#2196F3", hover_color="#1976D2").pack(side="bottom", anchor="e")

        def auto_close():
            try:
                if details_window.winfo_exists(): details_window.destroy()
            except: pass
        details_window.after(15000, auto_close)
        
        self.announce_student_details(name, program, batch)
        return details_window

    def announce_student_details(self, name: str, program: str, batch: str):
        announcement_text = f"Verification successful. Welcome, {name}. Program: {program}. Batch 20{batch}. Access granted."
        self.speak_text(announcement_text)

    def select_frame_by_name(self, name):
        self.verify_frame.grid_remove()
        self.logs_frame.grid_remove()
        if name == "verify":
            self.verify_frame.grid(row=0, column=1, sticky="nsew")
        elif name == "logs":
            self.logs_frame.grid(row=0, column=1, sticky="nsew")
            
    def verify_student_frame_event(self):
        self.select_frame_by_name("verify")

    def view_logs_frame_event(self):
        self.select_frame_by_name("logs")
        logs = self.system.view_logs()
        self.log_textbox.configure(state="normal")
        self.log_textbox.delete("1.0", "end")
        self.log_textbox.insert("1.0", logs)
        self.log_textbox.configure(state="disabled")

    def export_report_event(self):
        message = self.system.export_verification_report()
        messagebox.showinfo("Export Report", message)

    def check_timeouts(self):
        current_time = time.time()
        if self.verification_step == "qr_scan" and self.qr_scan_start_time:
            if current_time - self.qr_scan_start_time > self.max_qr_scan_time:
                self.handle_timeout("qr_scan")
                return True
        elif self.verification_step == "face_verify" and self.verification_start_time:
            if current_time - self.verification_start_time > self.max_verification_time:
                self.handle_timeout("face_verify")
                return True
        return False
    
    def handle_timeout(self, timeout_type):
        if timeout_type == "qr_scan":
            self.speak_text("QR code scan timeout.")
            messagebox.showwarning("Timeout", "QR code scanning timed out.")
        elif timeout_type == "face_verify":
            self.speak_text("Face verification timeout.")
            messagebox.showwarning("Timeout", "Face verification timed out.")

        if self.current_student_id:
            self.system.log_verification(
                self.current_student_id, self.current_student_name, self.current_student_program,
                False, datetime.now().isoformat(), score=self.best_score,
                verification_time=time.time() - self.verification_start_time if self.verification_start_time else 0
            )
        self.finalize_verification(False)
    
    def start_verification_process(self):
        if self.verification_active: return
        self.cap = cv2.VideoCapture(0)
        if not self.cap.isOpened():
            messagebox.showerror("Camera Error", "Cannot open camera.")
            return
        self.cap.set(cv2.CAP_PROP_FRAME_WIDTH, self.system.config.CAMERA_WIDTH)
        self.cap.set(cv2.CAP_PROP_FRAME_HEIGHT, self.system.config.CAMERA_HEIGHT)
        
        self.verification_active = True
        self.verification_step = "qr_scan"
        self.qr_scan_start_time = time.time()
        self.start_button.configure(state="disabled", text="Verification in Progress...")
        
        self.update_frame()

    def stop_verification_process(self):
        self.verification_active = False
        self.verification_step = "idle"
        if self.cap:
            self.cap.release()
            self.cap = None
        self.start_button.configure(state="normal", text="Start Verification")
        self.current_student_id = None
        self.consecutive_matches = 0
        self.best_score = 0.0
        self.last_known_face = None
        self.frame_count = 0
        self.progress_bar.set(0)

    def display_timeout_info(self):
        current_time = time.time()
        if self.verification_step == "qr_scan" and self.qr_scan_start_time:
            remaining = max(0, self.max_qr_scan_time - (current_time - self.qr_scan_start_time))
            self.info_label.configure(text=f"Position QR code in view. Time remaining: {remaining:.0f}s")
        elif self.verification_step == "face_verify" and self.verification_start_time:
            remaining = max(0, self.max_verification_time - (current_time - self.verification_start_time))
            self.info_label.configure(text=f"Face verification. Time: {remaining:.0f}s")

    def update_frame(self):
        if not self.verification_active: return
        if self.check_timeouts(): return 
    
        ret, frame = self.cap.read()
        if not ret:
            self.after(15, self.update_frame)
            return
        
        frame = cv2.flip(frame, 1)
        self.display_timeout_info()
        
        if self.verification_step == "qr_scan":
            self.handle_qr_scan(frame)
        elif self.verification_step == "face_verify":
            self.handle_face_verify(frame)
        
        img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(img)
        ctk_img = ctk.CTkImage(light_image=img, dark_image=img, size=(self.system.config.CAMERA_WIDTH, self.system.config.CAMERA_HEIGHT))
        self.camera_label.configure(image=ctk_img)
        self.camera_label.image = ctk_img
        self.after(15, self.update_frame)

    def handle_qr_scan(self, frame):
        self.status_label.configure(text="📱 Step 1: Scan Student QR Code")
        for obj in decode(frame):
            qr_data = obj.data.decode("utf-8")
            student_id, name, program = self.system.validate_qr_data(qr_data)
            
            if all([student_id, name, program]):
                self.current_student_id = student_id
                self.current_student_name = name
                self.current_student_program = program
                points = obj.polygon
                hull = cv2.convexHull(np.array([point for point in points], dtype=np.float32))
                cv2.polylines(frame, [np.int32(hull)], True, (0, 255, 0), 3)
                self.prepare_face_verification()
                return

    def prepare_face_verification(self):
        self.status_label.configure(text="⚙️ Preparing face verification...", text_color="yellow")
        self.info_label.configure(text=f"Student: {self.current_student_name} ({self.current_student_program})")
        
        # Train the student-specific LBPH model
        training_success = self.system.train_student_lbph_model(self.current_student_id)

        if not training_success:
            messagebox.showerror("Error", f"Could not process face data for {self.current_student_name}. Ensure enough quality photos are registered.")
            self.finalize_verification(False)
            return

        self.verification_step = "face_verify"
        self.verification_start_time = time.time()
        self.consecutive_matches = 0
        self.best_score = 0.0

    def handle_face_verify(self, frame):
        threshold_percent = self.system.config.VERIFICATION_SCORE_THRESHOLD * 100
        self.status_label.configure(text=f"📸 Step 2: Position Your Face Clearly (Threshold: {threshold_percent:.0f}%)")
        self.frame_count += 1
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        if self.frame_count % 3 == 0: # Detect less frequently to save resources
            faces = self.system.dlib_detector(gray, 0)
            if faces:
                self.last_known_face = max(faces, key=lambda rect: rect.width() * rect.height())
            else:
                self.last_known_face = None
    
        if self.last_known_face is not None:
            face = self.last_known_face
            x, y, w, h = face.left(), face.top(), face.width(), face.height()

            face_roi = gray[y:y+h, x:x+w]
            processed_face = self.system.preprocess_face(face_roi)
            
            label, confidence = self.system.lbph_recognizer.predict(processed_face)
            
            # Convert confidence (distance) to a similarity score (0.0 - 1.0)
            # A score of 1.0 is a perfect match (confidence=0)
            # A score of 0.0 is a match at the threshold boundary
            final_score = max(0.0, 1.0 - (confidence / self.system.config.LBPH_CONFIDENCE_THRESHOLD))

            self.best_score = max(self.best_score, final_score)
            self.progress_bar.set(final_score)

            if final_score > self.system.config.VERIFICATION_SCORE_THRESHOLD:
                self.consecutive_matches += 1
                status_text = f"MATCHING... {self.consecutive_matches}/{self.system.config.REQUIRED_CONSECUTIVE}"
                status_color = "#32CD32"
                cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
            else:
                self.consecutive_matches = 0
                status_text = "NO MATCH"
                status_color = "red"
                cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 2)
    
            score_text = f"Match: {final_score*100:.1f}%"
            cv2.putText(frame, score_text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

            self.info_label.configure(
                text=f"Score: {final_score*100:.1f}% | Best: {self.best_score*100:.1f}% | Status: {status_text}",
                text_color=status_color
            )
    
            if self.consecutive_matches >= self.system.config.REQUIRED_CONSECUTIVE:
                self.finalize_verification(True)
        else:
            self.info_label.configure(text="No face detected. Please ensure good lighting.", text_color="orange")
            self.progress_bar.set(0)

    def finalize_verification(self, success):
        runtime = time.time() - self.verification_start_time if self.verification_start_time else 0.0
        final_score = self.best_score
        
        timestamp = datetime.now().isoformat()
        student_id = self.current_student_id
        student_name = self.current_student_name
        student_program = self.current_student_program

        # Log even if student_id is None (e.g., from timeout before QR scan)
        self.system.log_verification(
            student_id or "UNKNOWN", student_name or "N/A", student_program or "N/A", 
            success, timestamp, score=final_score, verification_time=runtime
        )
        
        # Stop camera and reset state
        self.stop_verification_process()
        
        if success:
            self.status_label.configure(text="✅ ACCESS GRANTED", text_color="green")
            self.info_label.configure(text=f"Welcome, {student_name}! Final Score: {final_score*100:.1f}%", text_color="green")
            self.show_student_details_screen(student_id, final_score)
        else:
            self.status_label.configure(text="❌ ACCESS DENIED", text_color="red")
            self.info_label.configure(text=f"Face not recognized. Best score: {final_score*100:.1f}%", text_color="red")
            messagebox.showerror("Failure", "Face verification failed.")
            
        try:
            result_logger.save_result(
                member_name=self.member_name,
                student_id=student_id or "UNKNOWN",
                score=final_score,
                runtime=runtime
            )
        except Exception as e:
            print(f"Failed to log result: {e}")
            
        self.after(5000, self.reset_labels)
    
    def reset_labels(self):
        self.status_label.configure(text="Welcome! Click 'Start Verification' to begin.", text_color=("white", "black"))
        self.progress_bar.set(0)
        self.info_label.configure(text="")
        
    def on_closing(self):
        if self.verification_active: self.stop_verification_process()
        if self.cap and self.cap.isOpened(): self.cap.release()
        try:
            if hasattr(self, 'tts_system'): self.tts_system.cleanup()
        except Exception as e: print(f"TTS cleanup failed: {e}")
        self.quit()
        self.destroy()
        sys.exit(0)

if __name__ == "__main__":
    try:
        config = Config()
        backend_system = StudentSystem(config)
        app = App(student_system=backend_system)
        app.mainloop()
    except KeyboardInterrupt:
        print("\n👋 System interrupted by user. Goodbye!")
    except Exception as e:
        logger.critical(f"Critical system error on startup: {e}")
        print(f"💥 Critical error: {e}")

2025-08-30 09:13:24,660 - INFO - Loaded data for 2 students
2025-08-30 09:13:24,942 - INFO - Student Verification System backend initialized with dlib and LBPH
2025-08-30 09:13:25,170 - INFO - RobustTTS worker thread started.
2025-08-30 09:13:39,485 - INFO - QR ID 'S002' found in database for student: Kah Yung
2025-08-30 09:13:39,497 - INFO - Starting LBPH model training for student: S002
2025-08-30 09:13:39,532 - INFO - Loaded 30 face images for student S002
2025-08-30 09:13:40,156 - INFO - Training LBPH model with 25 face samples.
2025-08-30 09:13:40,224 - INFO - LBPH model for S002 trained successfully.
2025-08-30 09:13:46,574 - INFO - Verification logged: Kah Yung - SUCCESS (Score: 0.629)


✓ Result saved for Member1_Dlib_LBPH: Run #2, Score=0.629, Runtime=6.317s


2025-08-30 09:14:02,179 - INFO - Cleaning up TTS system...
2025-08-30 09:14:02,181 - INFO - TTS system cleaned up.


SystemExit: 0